In [81]:
import pandas as pd
import numpy as np
import tqdm
import json
import datetime

from catboost import Pool, CatBoostClassifier
from  sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, log_loss

In [161]:
# Юра
import pickle
# для w2v
import gensim
import os
from collections import Counter
from pymorphy2 import MorphAnalyzer

Slow version of gensim.models.doc2vec is being used


In [162]:
w2v_fpath = os.path.join('../eduhack/w2v/all.norm-sz100-w10-cb0-it1-min100.w2v')
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
w2v.init_sims(replace=True)

In [59]:
pd.set_option('max_columns', 200)

### Даные

In [3]:
data = pd.read_csv('user_info.csv', sep=';', encoding = 'cp1251')

/home/ubuntu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (40,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [164]:
# данные по группам
with open('data/data_group.pikle', 'rb') as f:
    data_group = pickle.load(f)

In [167]:
# о колонках
ids_data_group = ['uid']
label_data_group = ['company', 'position', 'target_text']
unuseble_data_group = ['group_all_data', 'post_all_data']
useble_data_group = ['group_count', 'post_count', 'group_activity','group_description', 'group_deactivated_part', 'group_is_closed_part',
'group_name', 'group_event_part', 'group_page_part', 'group_group_part','group_status', 'group_market_part', 
'group_fixed_post_mean','group_site_exist_part']

In [168]:
data_group[ids_data_group+useble_data_group]

,uid,group_count,post_count,group_activity,group_description,group_deactivated_part,group_is_closed_part,group_name,group_event_part,group_page_part,group_group_part,group_status,group_market_part,group_fixed_post_mean,group_site_exist_part
0,72047483,500.0,2.0,"[Обмен музыкой, Веб-сайт, Юмор, Юмор, Кинемато...","[Приятного прослушивания и спасибо, что вы под...",0.118000,0,"[Новинки Музыки 2017, Киномания | Новинки кино...",0.188000,0.428000,0.384000,[Подписывайтесь и наслаждайтесь музыкой вместе...,0.097506,3.306912e+05,0.394558
1,226482618,3.0,10.0,"[Веб-сайт, nan, Открытая группа]",[Ключи к Farming 2013:<br>AXSYS-THTL5-WHCJC-86...,0.333333,0,"[Моды для Farming Simulator 2015-2017, Kik Bet...",0.000000,0.333333,0.666667,"[, nan, ]",0.000000,5.196000e+03,0.500000
2,69429094,7.0,NaN,"[Закрытая группа, Открытая группа, Публичная с...",[Внимание! Наша группа НЕ является официальной...,0.000000,0,"[АЭРОФЛОТ - Российские Авиалинии, ВАУ-ОЛАГА-АГ...",0.000000,0.142857,0.857143,"[неофициальная группа, #spbguga #спбгуга, , , ...",0.000000,2.322000e+03,0.714286
3,64077853,NaN,NaN,[],[],0.000000,0,[],0.000000,0.000000,0.000000,[],0.000000,0.000000e+00,0.000000
4,101404683,NaN,NaN,[],[],0.000000,0,[],0.000000,0.000000,0.000000,[],0.000000,0.000000e+00,0.000000
5,43801966,NaN,NaN,[],[],0.000000,0,[],0.000000,0.000000,0.000000,[],0.000000,0.000000e+00,0.000000
6,161183513,1.0,NaN,[Поэт],[Страница персидского поэта Гияса ад-Дина Абу-...,0.000000,0,[Омар Хайям],0.000000,1.000000,0.000000,"[И другие великие философы, притчи, саморазвит...",0.000000,0.000000e+00,1.000000
7,218939119,1.0,5.0,[Туризм и отдых],[Всё всё всё о любимом Египте. Добро пожаловат...,0.000000,0,[ЕГИПЕТ - I ♥ Egypt],0.000000,1.000000,0.000000,[Everything about Egypt!!! Lovely places!],0.000000,3.528600e+04,1.000000
8,289436419,NaN,20.0,[],[],0.000000,0,[],0.000000,0.000000,0.000000,[],0.000000,0.000000e+00,0.000000
9,90894209,NaN,20.0,[],[],0.000000,0,[],0.000000,0.000000,0.000000,[],0.000000,0.000000e+00,0.000000


## Предобработка

In [4]:
for index, row in tqdm.tqdm(data.iterrows(), total = data.shape[0]):
    
    # career
    if not pd.isnull(row.career):
        
        career_str = row.career.replace('"', '').replace('\'', '"')
        
        # если работ несколько - возьмем последнюю
        split_index = career_str.rfind(', {')
        if (split_index != -1):
            career_str = career_str[:2] + career_str[split_index + 3:]
            
        try:
            career_list = json.loads(career_str)
        except json.JSONDecodeError:
            continue
        
        career_df = pd.DataFrame(career_list)
            
        # записываем все поля что есть
        for column in career_df.columns.values:
            data.loc[index, 'career_' + column] = career_df[column].values[0]
    
    # personal
    if not pd.isnull(row.personal):
        
        personal_str = row.personal.replace('"', '').replace('\'', '"')
        
        split_index = personal_str.rfind(', {')
        if (split_index != -1):
            personal_str = personal_str[:2] + personal_str[split_index + 3:]
            
        try:
            personal_list = json.loads(personal_str)
        except json.JSONDecodeError:
            continue
        
        try:
            personal_df = pd.DataFrame(personal_list)
        except ValueError:
            personal_df = pd.DataFrame(personal_list, index = [0])
            
            
        # записываем все поля что есть
        for column in personal_df.columns.values:
            
            if column == 'langs':
                
                langs = personal_df[column].values
                lang_count =0 
                for lang in langs:
                    
                    if lang.lower() in ['русский', 'english', 'українська', 'романи', 'dangme', 'deutsch', 'italiano', 'polski', 'беларуский']:
                        data.loc[index, 'personal_lang_' + str(lang)] = 1
                        lang_count += 1
                    
                data.loc[index, 'personal_lang_count'] = lang_count
                
            else:            
                data.loc[index, 'personal_' + column] = personal_df[column].values[0]
    
    
    # schools
    if not pd.isnull(row.schools):
        
        schools_str = row.schools.replace('"', '').replace('\'', '"')
        
        split_index = schools_str.rfind(', {')
        if (split_index != -1):
            schools_str = schools_str[:2] + schools_str[split_index + 3:]
            
        try:
            schools_list = json.loads(schools_str)
        except json.JSONDecodeError:
            continue
        
        schools_df = pd.DataFrame(schools_list)
            
        # записываем все поля что есть
        for column in schools_df.columns.values:
            data.loc[index, 'school_' + column] = schools_df[column].values[0]
    
    
    # universities
    if not pd.isnull(row.universities):
        
        universities_str = row.universities.replace('"', '').replace('\'', '"')
        
        split_index = universities_str.rfind(', {')
        if (split_index != -1):
            universities_str = universities_str[:2] + universities_str[split_index + 3:]
            
        try:
            universities_list = json.loads(universities_str)
        except json.JSONDecodeError:
            continue
        
        universities_df = pd.DataFrame(universities_list)
            
        # записываем все поля что есть
        for column in universities_df.columns.values:
            data.loc[index, 'universities_' + column] = universities_df[column].values[0]

100%|██████████| 116117/116117 [28:43<00:00, 67.36it/s]


In [ ]:
for column in data.columns.values:
    if 'personal_lang' in column:
        data[column].fillna(0, inplace = True)

In [ ]:
# data.bdate = pd.DatetimeIndex(data.bdate)

# data['month_born'] = pd.DatetimeIndex(data['bdate'].values, format='%d.%m.%y').month 
# data['year_born'] = pd.DatetimeIndex(data['bdate'].values, format='%d.%m.%y').year

In [75]:
# data.to_csv('user_info_prepared.csv', sep=';', index = False)

In [28]:
data = pd.read_csv('user_info_prepared.csv', sep=';')

/home/ubuntu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (40,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Target

In [16]:
target = pd.read_csv('data/target.csv', sep=';', encoding = 'cp1251').dropna()
target = target[['uid', 'target_text']]
target.columns = ['uid', 'y']

target_dict = {'Прочее':0, 'Экономика и управление': 1, 'Техника и технологи': 2, 'Образование и педагогика':3, 'Здравоохранение':4,
              'Культура и искусство':5, 'Гуманитарные и социальные науки':6, 'Юридические науки':7, 'Естественные науки и метематика':8, 'Сельское и рыбное хозяйство':9}

# make target as int
target.y = target.y.map(lambda x: target_dict.get(x))
target.y = target.y.astype('int32')

data = data.merge(target, on = ['uid'], how = 'left')

### Добавляю данные по группам

In [172]:
new_df = data.merge(data_group[ids_data_group+useble_data_group], how='left', on='uid')

### w2v кусок

In [176]:
text_data = ['about', 'books', 'games', 'interests', 'movies', 'music', 'quotes', 'status', 'tv',
             'universities_faculty_name', 'universities_chair_name', 'personal_religion', 'school_speciality']
# названия столбцов , для которых нужен w2v
w2v_columns = ['group_activity','group_description','group_name','group_status']

In [197]:
# Для каждого Serias должен быть свой следующий обхект
counter_dict = {}
for i in w2v_columns+text_data:
    # число документов, в которые входит каждое слово
    counter_dict[i]=Counter([w for text in new_df[i].map(joiner) for w in set(text.split())])

In [174]:
# множество всех слов, входящих в word2vec
vocab = set(w2v.index2word)

In [184]:
# убрать лишние символы, привести к нижнему регистру
def strip_words(sentence):
    # приводим к нижнему регистру
    sentence = sentence.lower()
    # цикл по каждому символу
    for ch in '.,()!\/#=-0123456789:;?"':
        # заменяем их на пробелы
        sentence = sentence.replace(ch, ' ')
    return sentence

# для объединения массива слов/названий и иже в один текст
def joiner(array):
    try:
        return(strip_words(' '.join(pd.Series(array)[-pd.Series(array).isnull()].values)))
    except: return('')
# Делаю для одного Serias
def text2vec(lemmas, total_df): #total_df - словарь Counter
    """ Turns text into a vector
        Requires an instance of w2v gensim model and an instance of total_df
    """
    #print(words)
    total = np.zeros(w2v.syn0.shape[1]) # сумма векторов
    weights = 1e-4 # сумма весов всех векторов
    for word in lemmas.split():
        word = word.replace('ё', 'е').strip()
        if word in vocab:
            v = w2v.word_vec(word)
            # часто глобально встречающиеся слова мы взвешиваем вниз
            w = 1.0 / np.log(2+total_df[word])
            total = total + v * w
            weights = weights + w
        else:
            pass
    ans = total / weights
    # нормируем вектор, чтобы он имел единичную длину, если в нём вообще есть слагаемые
    size = sum(ans**2)
    if size > 1e-4:
        ans = ans / np.sqrt(size)
    return ans

In [200]:
# Для всех столбцов, названия которых есть в w2v_columns создаю доп столбец, который кодирует список в w2v вектор
for i in w2v_columns+text_data:
    new_df[i+'_enc'] = new_df[i].map(joiner).map(lambda x: text2vec(x,counter_dict[i]))

In [201]:
len(w2v_columns+text_data)

17

## add new_df

In [207]:
new_df_back = new_df.copy()

In [208]:
for column in new_df.columns.values:
    
    if '_enc' in column:
        
        new_df[column] = new_df[column].map(lambda x: np.mean(x))

In [219]:
cos = list(new_df.loc[:, 'group_count':].columns.values)
cos.append("uid")
data = data.merge(new_df.loc[: , cos], on = ['uid'], how = 'left')

### Model

In [359]:
columns = ['blacklisted_by_me', 'can_see_all_posts', 'can_send_friend_request', 'city', 'faculty', 'graduation', 'has_photo', 'relation', 'wall_comments', 'universities_country', 'universities_graduation', 'universities_faculty', 'personal_lang_Русский', 'personal_lang_English', 'personal_alcohol', 'personal_people_main', 'personal_smoking', 'personal_lang_Романи', 'personal_lang_Italiano', 'school_city', 'school_country', 'school_type', 'school_year_graduated' ,'career_city_id', 'career_group_id', 'career_until']

In [349]:
columns = ['blacklisted', 'blacklisted_by_me',
          'can_post', 'can_see_all_posts', 'can_see_audio', 'can_send_friend_request', 'can_write_private_message',
          'city', 'country',
          'faculty', 'followers_count',
          'graduation', 'has_mobile', 'has_photo', 'hidden',
          'relation', 'sex', 'wall_comments',
          'universities_city', 'universities_country', 'universities_graduation', 'universities_id', 'universities_faculty', 'universities_chair',
          'personal_lang_Русский', 'personal_lang_Українська', 'personal_lang_English', 'personal_lang_count', 'personal_alcohol',
          'personal_life_main', 'personal_people_main', 'personal_political', 'personal_smoking', 'personal_lang_Deutsch', 'personal_lang_Романи', 'personal_lang_Dangme', 'personal_lang_Italiano', 'personal_lang_Polski',
          'school_city', 'school_country', 'school_id', 'school_type', 'school_year_from', 'school_year_graduated', 'school_year_to',
          'career_city_id', 'career_country_id', 'career_from', 'career_until']

#           'group_count', 'post_count', 'group_deactivated_part', 'group_is_closed_part', 'group_event_part', 'group_page_part', 'group_group_part', 'group_market_part', 'group_fixed_post_mean', 'group_site_exist_part',
#           'group_activity_enc', 'group_description_enc', 'group_name_enc', 'group_status_enc', 'about_enc', 'books_enc', 'games_enc', 'interests_enc', 'movies_enc','music_enc', 'quotes_enc', 'status_enc', 'tv_enc', 'universities_faculty_name_enc', 'universities_chair_name_enc', 'personal_religion_enc','school_speciality_enc']

# for column in columns:
#     if data[column].dtype == 'object':
#         data[column].fillna("", inplace = True)


In [378]:
X_train, X_test, y_train, y_test = train_test_split(data[~pd.isnull(data.y)][columns], data[~pd.isnull(data.y)]['y'], stratify = data[~pd.isnull(data.y)]['y'], test_size= 0.2, random_state = 42)

In [376]:
cb = CatBoostClassifier(loss_function='MultiClassOneVsAll', classes_count = data['y'].unique().shape[0])
cb.fit(X_train, y_train
#        , cat_features = [columns.index('education_form'), columns.index('education_status'), columns.index('home_town'), columns.index('universities_education_status'), columns.index('universities_education_form'), columns.index('personal_inspired_by'), columns.index('school_class')]
      )

predictions = cb.predict(X_test, y_test)

In [381]:
predictions = cb.predict(data[~pd.isnull(data.y)][columns])

print('micro')
print('accuracy_score', accuracy_score(predictions, data[~pd.isnull(data.y)]['y']))
print('precision_score', precision_score(predictions, data[~pd.isnull(data.y)]['y'], average='micro'))
print('recall_score', recall_score(predictions, data[~pd.isnull(data.y)]['y'], average='micro'))
print('f1_score', f1_score(predictions, data[~pd.isnull(data.y)]['y'], average='micro'))

print()
print('macro')
print('accuracy_score', accuracy_score(predictions, data[~pd.isnull(data.y)]['y']))
print('precision_score', precision_score(predictions, data[~pd.isnull(data.y)]['y'], average='macro'))
print('recall_score', recall_score(predictions, data[~pd.isnull(data.y)]['y'], average='macro'))
print('f1_score', f1_score(predictions, data[~pd.isnull(data.y)]['y'], average='macro'))

print()
print('weighted')
print('accuracy_score', accuracy_score(predictions, data[~pd.isnull(data.y)]['y']))
print('precision_score', precision_score(predictions, data[~pd.isnull(data.y)]['y'], average='weighted'))
print('recall_score', recall_score(predictions, data[~pd.isnull(data.y)]['y'], average='weighted'))
print('f1_score', f1_score(predictions, data[~pd.isnull(data.y)]['y'], average='weighted'))

micro
accuracy_score 0.474033580633
precision_score 0.474033580633
recall_score 0.474033580633
f1_score 0.474033580633

macro
accuracy_score 0.474033580633
precision_score 0.214954837324
recall_score 0.57325713061
f1_score 0.210674015255

weighted
accuracy_score 0.474033580633
precision_score 0.692340091553
recall_score 0.474033580633
f1_score 0.53985677316


/home/ubuntu/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/ubuntu/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Сохраним модель

In [363]:
scolumn

['blacklisted_by_me',
 'can_see_all_posts',
 'can_send_friend_request',
 'city',
 'faculty',
 'graduation',
 'has_photo',
 'relation',
 'wall_comments',
 'universities_country',
 'universities_graduation',
 'universities_faculty',
 'personal_lang_Русский',
 'personal_lang_English',
 'personal_alcohol',
 'personal_people_main',
 'personal_smoking',
 'personal_lang_Романи',
 'personal_lang_Italiano',
 'school_city',
 'school_country',
 'school_type',
 'school_year_graduated',
 'career_city_id',
 'career_group_id',
 'career_until',
 'uid',
 'uid']

In [364]:
scolumn = columns
scolumn.append('uid')
data[scolumn][:500].to_csv('results/model_user_info.csv', sep=';', index = False)

In [361]:
cb.fit(data[~pd.isnull(data.y)][columns], data[~pd.isnull(data.y)]['y'])
cb.save_model('results/cb_model')

In [366]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [368]:
precision_score?

In [374]:
predictions = cb.predict(data[~pd.isnull(data.y)][columns])

print('accuracy_score', accuracy_score(predictions, data[~pd.isnull(data.y)]['y']))
print('precision_score', precision_score(predictions, data[~pd.isnull(data.y)]['y'], average='micro'))
print('recall_score', recall_score(predictions, data[~pd.isnull(data.y)]['y'], average='micro'))
print('f1_score', f1_score(predictions, data[~pd.isnull(data.y)]['y'], average='micro'))

accuracy_score 0.479890667708
precision_score 0.479890667708
recall_score 0.479890667708
f1_score 0.479890667708


In [365]:
data[scolumn][:500][~pd.isnull(data['personal_lang_Русский'])]

/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,blacklisted_by_me,can_see_all_posts,can_send_friend_request,city,faculty,graduation,has_photo,relation,wall_comments,universities_country,universities_graduation,universities_faculty,personal_lang_Русский,personal_lang_English,personal_alcohol,personal_people_main,personal_smoking,personal_lang_Романи,personal_lang_Italiano,school_city,school_country,school_type,school_year_graduated,career_city_id,career_group_id,career_until,uid,uid,uid
0,0.0,1.0,1.0,610.0,0.0,2006.0,0,0.0,1.0,3.0,2006.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,610.0,0.0,1.0,2001.0,NaN,NaN,NaN,26321178,26321178,26321178
1,0.0,0.0,1.0,1.0,0.0,0.0,1,0.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,282343969,282343969,282343969
2,0.0,1.0,1.0,392.0,0.0,0.0,1,4.0,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,392.0,3.0,NaN,2005.0,NaN,NaN,NaN,166935847,166935847,166935847
3,0.0,0.0,1.0,151.0,NaN,NaN,1,NaN,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13955970,13955970,13955970
4,0.0,1.0,1.0,2086.0,0.0,0.0,1,0.0,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,2086.0,1.0,NaN,2019.0,NaN,NaN,NaN,344820263,344820263,344820263
5,0.0,1.0,1.0,284.0,0.0,0.0,1,6.0,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,284.0,4.0,13.0,2021.0,NaN,NaN,NaN,237909561,237909561,237909561
6,0.0,0.0,1.0,1.0,NaN,NaN,1,NaN,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102725951,102725951,102725951
7,0.0,1.0,1.0,151.0,NaN,NaN,1,NaN,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64493273,64493273,64493273
8,0.0,1.0,1.0,1.0,0.0,0.0,0,0.0,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,NaN,1995.0,NaN,NaN,NaN,229142258,229142258,229142258
9,0.0,1.0,1.0,1496.0,0.0,0.0,1,1.0,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1496.0,1.0,NaN,NaN,NaN,NaN,NaN,296625301,296625301,296625301
